# Importing Libraries

In [1]:
#Multi-class Classification
import zipfile
import nibabel as nib
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.utils import make_grid
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import glob
from torch.autograd import Variable
from torchvision import models
from torch.nn import Module,  Linear, ReLU, CrossEntropyLoss, Sequential, Conv3d, MaxPool3d, Softmax, BatchNorm3d
from torch.optim import Adam
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import seaborn as sn
import torch.optim as optim
from torchsummary import summary
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import os
#walk through directory and list through files
for dirpath, dirnames, filenames in os.walk("/content/drive/My Drive/ADNI_full"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")
len(filenames)


There are 4 directories and 0 images in '/content/drive/My Drive/ADNI_full'.
There are 0 directories and 189 images in '/content/drive/My Drive/ADNI_full/AD'.
There are 0 directories and 220 images in '/content/drive/My Drive/ADNI_full/LMCI'.
There are 0 directories and 254 images in '/content/drive/My Drive/ADNI_full/CN'.
There are 0 directories and 0 images in '/content/drive/My Drive/ADNI_full/.ipynb_checkpoints'.


0

In [4]:
#Get the class names
import numpy as np
import pathlib
data_dir = pathlib.Path('/content/drive/My Drive/ADNI_full')
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)

['.ipynb_checkpoints' 'AD' 'CN' 'LMCI']


# Loading AD Images

In [5]:
AD = glob.glob(r'/content/drive/My Drive/ADNI_full/AD/*.nii.gz')
ADFiles = AD
ADnumfiles = len(ADFiles)



x_range_from = 10; x_range_to = 110
y_range_from = 40; y_range_to = 95
z_range_from = 5; z_range_to = 105
ADimages = np.zeros((ADnumfiles, z_range_to-z_range_from, x_range_to-x_range_from, y_range_to-y_range_from, 1), 
                  dtype=np.float32) # numfiles× z × x × y ×1; avoid 64bit types
print(ADimages.shape)

# Actually load nifti files into array
for i in range(ADnumfiles):   # for loop over files and load
    if (i % 20 == 0):
        print('Loading file %d of %d' % (i+1, ADnumfiles))
    ADimg = nib.load(ADFiles[i])
    ADimg = ADimg.get_fdata()[x_range_from:x_range_to, y_range_from:y_range_to, z_range_from:z_range_to]
    ADimg = np.transpose(ADimg, (2, 0, 1)) # reorder dimensions to match coronal view z*x*y in MRIcron etc.
    #print(img.shape)
    ADimg = np.flip(ADimg)
    ADimages[i, :,:,:, 0] = np.nan_to_num(ADimg)
print('Successfully loaded files')
print('Image array size: ', ADimages.shape)

#y_label for AD
labelAD=np.full(len(ADimages),1)
print('LabelAD size: ', labelAD.shape)


Found  189  nifti files
(189, 100, 100, 55, 1)
Loading file 1 of 189
Loading file 21 of 189
Loading file 41 of 189
Loading file 61 of 189
Loading file 81 of 189
Loading file 101 of 189
Loading file 121 of 189
Loading file 141 of 189
Loading file 161 of 189
Loading file 181 of 189
Successfully loaded files
Image array size:  (189, 100, 100, 55, 1)
LabelAD size:  (189,)
LabelADD size:  (189,)


# Loading CN Images

In [6]:
CN = glob.glob(r'/content/drive/My Drive/ADNI_full/CN/*.nii.gz')
CNFiles = CN
CNnumfiles = len(CNFiles)



x_range_from = 10; x_range_to = 110
y_range_from = 40; y_range_to = 95
z_range_from = 5; z_range_to = 105
CNimages = np.zeros((CNnumfiles, z_range_to-z_range_from, x_range_to-x_range_from, y_range_to-y_range_from, 1), 
                  dtype=np.float32) # numfiles× z × x × y ×1; avoid 64bit types
print(CNimages.shape)

# Actually load nifti files into array
for i in range(CNnumfiles):   # for loop over files and load
    if (i % 20 == 0):
        print('Loading file %d of %d' % (i+1, CNnumfiles))
    CNimg = nib.load(CNFiles[i])
    CNimg = CNimg.get_fdata()[x_range_from:x_range_to, y_range_from:y_range_to, z_range_from:z_range_to]
    CNimg = np.transpose(CNimg, (2, 0, 1)) # reorder dimensions to match coronal view z*x*y in MRIcron etc.
    CNimg = np.flip(CNimg) # flip all positions
    #print(img.shape)
    CNimages[i, :,:,:, 0] = np.nan_to_num(CNimg)
print('Successfully loaded files')
print('Image array size: ', CNimages.shape)
#y_label for CN
labelCN=np.full(len(CNimages),0)
print('LabelCN size: ', labelCN.shape)


Found  254  nifti files
(254, 100, 100, 55, 1)
Loading file 1 of 254
Loading file 21 of 254
Loading file 41 of 254
Loading file 61 of 254
Loading file 81 of 254
Loading file 101 of 254
Loading file 121 of 254
Loading file 141 of 254
Loading file 161 of 254
Loading file 181 of 254
Loading file 201 of 254
Loading file 221 of 254
Loading file 241 of 254
Successfully loaded files
Image array size:  (254, 100, 100, 55, 1)
LabelCN size:  (254,)
LabelCNN size:  (254,)


In [ ]:
#Concatenation of training data
X=np.concatenate((CNimages, ADimages), axis=0)
print('x data of train size: ', X.shape)
y=np.concatenate((labelCN,labelAD), axis=0)
print('y data in  of train size: ', y.shape)

In [11]:
with open("Images_labels.txt", "wb") as fp:   #Pickling
    pickle.dump(X, fp)
    pickle.dump(y, fp)